# Predict breast cancer type using SageMaker linear-learner

Contents
- [0. Setup](#setup)
- [1. Introduction](#introduction)
- [2. Load and explore data](#load)
- [3. Create logistic regression model using SageMaker linear-learner algorithm](#model)
- [4. Deploy the model in the AWS Cloud](#deployment)
- [5. Score the model](#score)

**Note:** This notebook works correctly with kernel `Python 3.5+`.

<a id="setup"></a>
## 0. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

- Create a SageMaker Service, setup steps are described here: https://docs.aws.amazon.com/sagemaker/latest/dg/gs-set-up.html
- Install reqiured python packages from PyPi repository

### Package installation

In [52]:
!pip install boto3 | tail -n 1
!pip install sagemaker | tail -n 1
!pip install pandas | tail -n 1
!pip install scikit-learn | tail -n 1

<a id="introduction"></a>
## 1. Introduction

This notebook defines, trains, and deploys the model that predicts cancer type.

<a id="load"></a>
## 2. Load and explore data

In this section you will load data into a pandas DataFrame and perform a basic exploration. Next you will upload training data to the Amazon S3 Object Storage.

### 2.1 Load data from webpage

In [53]:
import pandas as pd
from sklearn.utils import shuffle

In [54]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header = None)

data.columns = ["id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean",
                "compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean",
                "radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se",
                "concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst",
                "perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst",
                "concave points_worst","symmetry_worst","fractal_dimension_worst"] 

### 2.2 Explore data

In [55]:
print("Sample records:")
display(data.head())

print("Features columns summary:")
display(data.iloc[:, 2:].describe())

print("Label column summary:")
display(data.diagnosis.value_counts())

Sample records:


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Features columns summary:


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


Label column summary:


B    357
M    212
Name: diagnosis, dtype: int64

### 2.3 Store training data in S3 Object Storage

You will use SageMaker linear-learner built-in algorithm. This algorithm expects the first column to be the label when training data is in `text/csv` format.

Moreover, the label column has to be numeric.

#### Save prepared data to local filesystem

In [56]:
data_shuffled = shuffle(data)
data_shuffled.replace({'diagnosis': {'M': 1, 'B': 0}}, inplace = True)
display(data_shuffled.head())

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
435,908489,1,13.98,19.62,91.12,599.5,0.10600,0.11330,0.11260,0.064630,...,17.04,30.80,113.90,869.3,0.16130,0.3568,0.4069,0.18270,0.3179,0.10550
231,88199202,0,11.32,27.08,71.76,395.7,0.06883,0.03813,0.01633,0.003125,...,12.08,33.75,79.82,452.3,0.09203,0.1432,0.1089,0.02083,0.2849,0.07087
497,914580,0,12.47,17.31,80.45,480.1,0.08928,0.07630,0.03609,0.023690,...,14.06,24.34,92.82,607.3,0.12760,0.2506,0.2028,0.10530,0.3035,0.07661
366,9011494,1,20.20,26.83,133.70,1234.0,0.09905,0.16690,0.16410,0.126500,...,24.19,33.81,160.00,1671.0,0.12780,0.3416,0.3703,0.21520,0.3271,0.07632
536,91979701,1,14.27,22.55,93.77,629.8,0.10380,0.11540,0.14630,0.061390,...,15.29,34.27,104.30,728.3,0.13800,0.2733,0.4234,0.13620,0.2698,0.08351


In [57]:
train_data_filename = 'breast_cancer.csv'
data_shuffled.iloc[:, 1:].to_csv(path_or_buf = train_data_filename, index = False, header = False)

**Note:** Header row has to be omitted.

#### Upload data to S3 Object Storage

In [58]:
import time
import json
import boto3

In [63]:
aws_credentials = {'access_key': '****',
                   'secret_key': '****',
                   'region_name': '****'} #i.e. us-east-2

**Note:** You have to provide credentials from your Amazon account.

In [64]:
session = boto3.Session(
    aws_access_key_id = aws_credentials['access_key'],
    aws_secret_access_key = aws_credentials['secret_key'],
    region_name = aws_credentials['region_name']
)
s3 = session.resource('s3')

#### 2.4 [Create an S3 bucket](https://s3.console.aws.amazon.com/s3) and use the name in the cell below for `bucket_name`.



In [67]:
bucket_name = 'XXXXXXXXX'
train_data_filename = 'breast_cancer.csv'
train_data_path = 'breast-cancer/train'
output_data_path = 's3://{}/breast-cancer/output'.format(bucket_name)
time_suffix = time.strftime("%Y-%m-%d-%H-%M", time.gmtime())

**Note:** You have to replace `bucket_name` with name of bucket in your S3 Object Storage. 

You can run following code `[bkt.name for bkt in s3.buckets.all()]` to list all your buckets.

In [68]:
s3_bucket = s3.Bucket(bucket_name)
s3_bucket.upload_file(Filename = train_data_filename, Key = '{}/{}'.format(train_data_path, train_data_filename))

Let's check if your data has been uploaded successfully.

In [69]:
for s3_obj in s3_bucket.objects.all():
    if (s3_obj.bucket_name == bucket_name) and (train_data_path in s3_obj.key):
        train_data_uri = 's3://{}/{}'.format(s3_obj.bucket_name, s3_obj.key)
        print(train_data_uri)

s3://scottda-sagemaker-us-east-2/breast-cancer/train/breast_cancer.csv


<a id="model"></a>
## 3. Create logistic regression model using SageMaker linear-learner algorithm

In this section you will learn how to:

- [3.1 Setup training parameters](#prep)
- [3.2 Start training job](#train)

<a id="prep"></a>
### 3.1 Setup training parameters

In [70]:
from sagemaker.amazon.amazon_estimator import get_image_uri

sm_client = session.client('sagemaker')

In [71]:
training_image = get_image_uri(session.region_name, 'linear-learner')

iam_client = session.client('iam')
[role_arn, *_] = [role['Arn'] for role in iam_client.list_roles()['Roles'] if 'AmazonSageMaker-ExecutionRole' in role['RoleName']]

linear_job_name = 'Breast-cancer-linear-learner-' + time_suffix

In [72]:
linear_training_params = {
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "HyperParameters": {
        "feature_dim": "30",
        "mini_batch_size": "100",
        "predictor_type": "binary_classifier",
        "epochs": "10",
        "num_models": "32",
        "loss": "auto"
    },
    "InputDataConfig": [{
        "ChannelName": "train",
        "ContentType": "text/csv", 
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": train_data_uri,
                "S3DataDistributionType": "ShardedByS3Key"
            }
        }
    }],
    "OutputDataConfig": {"S3OutputPath": output_data_path},
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.c4.xlarge",
        "VolumeSizeInGB": 2
    },
    "RoleArn": role_arn,
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 6 * 60
    },
    "TrainingJobName": linear_job_name

}

<a id="train"></a>
### 3.2 Start training job

In [73]:
sm_client.create_training_job(**linear_training_params)

{'TrainingJobArn': 'arn:aws:sagemaker:us-east-2:014862798213:training-job/breast-cancer-linear-learner-2019-01-28-16-13',
 'ResponseMetadata': {'RequestId': '2258ff91-9168-4f85-82af-3f97d86e1c5c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2258ff91-9168-4f85-82af-3f97d86e1c5c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '120',
   'date': 'Mon, 28 Jan 2019 16:14:01 GMT'},
  'RetryAttempts': 0}}

In [74]:
try:
    sm_client.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName = linear_job_name)
except Exception:
    print('Traing job error.')

In [75]:
train_job_details = sm_client.describe_training_job(TrainingJobName = linear_job_name)
train_job_status = train_job_details['TrainingJobStatus']

if train_job_status == 'Failed':
    print(train_job_details['FailureReason'])
else:
    train_job_arn = train_job_details['TrainingJobArn']
    print(train_job_arn)
    trained_model_uri = train_job_details['ModelArtifacts']['S3ModelArtifacts']
    print(trained_model_uri)

arn:aws:sagemaker:us-east-2:014862798213:training-job/breast-cancer-linear-learner-2019-01-28-16-13
s3://scottda-sagemaker-us-east-2/breast-cancer/output/Breast-cancer-linear-learner-2019-01-28-16-13/output/model.tar.gz


<a id="deployment"></a>
## 4. Deploy model in the AWS Cloud

### 4.1 Setup deployment parameters

In [76]:
linear_hosting_container = {'Image': training_image, 'ModelDataUrl': trained_model_uri}

create_model_details = sm_client.create_model(
    ModelName = linear_job_name,
    ExecutionRoleArn = role_arn,
    PrimaryContainer = linear_hosting_container)

print(create_model_details['ModelArn'])

arn:aws:sagemaker:us-east-2:014862798213:model/breast-cancer-linear-learner-2019-01-28-16-13


### 4.2 Create deployment configuration endpoint

In [77]:
endpoint_config = 'Breast-cancer-linear-endpoint-config-' + time_suffix
print(endpoint_config)

create_endpoint_config_details = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config,
    ProductionVariants = [{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': linear_job_name,
        'VariantName': 'AllTraffic'}])

Breast-cancer-linear-endpoint-config-2019-01-28-16-13


In [78]:
endpoint_config_details = sm_client.describe_endpoint_config(EndpointConfigName = endpoint_config)
print(endpoint_config_details)

{'EndpointConfigName': 'Breast-cancer-linear-endpoint-config-2019-01-28-16-13', 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:014862798213:endpoint-config/breast-cancer-linear-endpoint-config-2019-01-28-16-13', 'ProductionVariants': [{'VariantName': 'AllTraffic', 'ModelName': 'Breast-cancer-linear-learner-2019-01-28-16-13', 'InitialInstanceCount': 1, 'InstanceType': 'ml.m4.xlarge', 'InitialVariantWeight': 1.0}], 'CreationTime': datetime.datetime(2019, 1, 28, 9, 19, 5, 140000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'e340d068-2fdb-4da9-9b36-6034b5eec907', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e340d068-2fdb-4da9-9b36-6034b5eec907', 'content-type': 'application/x-amz-json-1.1', 'content-length': '436', 'date': 'Mon, 28 Jan 2019 16:19:07 GMT'}, 'RetryAttempts': 0}}


### 4.3 Create scoring endpoint

In [80]:
scoring_endpoint = 'Breast-cancer-endpoint-scoring-' + time_suffix

create_endpoint_details = sm_client.create_endpoint(
    EndpointName = scoring_endpoint,
    EndpointConfigName = endpoint_config)

In [81]:
try:
    sm_client.get_waiter('endpoint_in_service').wait(EndpointName = scoring_endpoint)
except Exception:
    print('Create scoring endpoint error')

In [82]:
scoring_endpoint_details = sm_client.describe_endpoint(EndpointName = scoring_endpoint)
scoring_enpoint_config_status = scoring_endpoint_details['EndpointStatus']
if scoring_enpoint_config_status != 'InService':
    print(scoring_endpoint_details['FailureReason'])
else:
    print(scoring_endpoint_details['EndpointArn'])

arn:aws:sagemaker:us-east-2:014862798213:endpoint/breast-cancer-endpoint-scoring-2019-01-28-16-13


<a id="score"></a>
## 5. Score the model

### 5.1 Prepare sample data for scoring

You will use data in `csv` format as the scoring payload. The first column (label) is removed from the data and the last 20 training records are selected.

In [83]:
scoring_data_filename = 'scoring_breast_cancer.csv'

In [84]:
with open(train_data_filename) as f_train:
    with open(scoring_data_filename, 'w') as f_score:
        f_score.writelines([','.join(line.split(',')[1:]) for line in f_train.readlines()[-20:]])

### 5.2 Send data for scoring

In [85]:
sm_runtime = session.client('runtime.sagemaker')

with open(scoring_data_filename) as f_payload:
    scoring_response = sm_runtime.invoke_endpoint(EndpointName = scoring_endpoint,
                                                  ContentType = 'text/csv',
                                                  Body = f_payload.read().encode())
    
    scored_records = scoring_response['Body'].read().decode()
    print(json.loads(scored_records))

{'predictions': [{'score': 8.901956221052387e-07, 'predicted_label': 0.0}, {'score': 6.122139893705025e-05, 'predicted_label': 0.0}, {'score': 0.9999994039535522, 'predicted_label': 1.0}, {'score': 0.03335440903902054, 'predicted_label': 0.0}, {'score': 0.9939921498298645, 'predicted_label': 1.0}, {'score': 1.0, 'predicted_label': 1.0}, {'score': 1.2264197266631527e-06, 'predicted_label': 0.0}, {'score': 0.00875077210366726, 'predicted_label': 0.0}, {'score': 0.0003120750479865819, 'predicted_label': 0.0}, {'score': 0.03921683132648468, 'predicted_label': 0.0}, {'score': 0.9999997615814209, 'predicted_label': 1.0}, {'score': 0.19013828039169312, 'predicted_label': 0.0}, {'score': 0.0038585402071475983, 'predicted_label': 0.0}, {'score': 0.31553658843040466, 'predicted_label': 0.0}, {'score': 1.4013206737217843e-06, 'predicted_label': 0.0}, {'score': 5.1617742428788915e-05, 'predicted_label': 0.0}, {'score': 8.58581472584774e-09, 'predicted_label': 0.0}, {'score': 2.9979563009874255e-07

### Authors

Wojciech Sobala, Data Scientist at IBM